In [10]:
!git clone https: // github.com / facebookresearch / fastText.git
!pip install fastText
!pip install datasets

fatal: destination path 'fastText' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 4.1 MB/s 
     |████████████████████████████████| 115 kB 59.2 MB/s 
     |████████████████████████████████| 212 kB 43.3 MB/s 
     |████████████████████████████████| 120 kB 37.3 MB/s 
     |████████████████████████████████| 127 kB 55.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [1]:
import fasttext
from datasets import load_dataset

dataset_train = load_dataset('imdb', split='train')

Reusing dataset imdb (/home/victor/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


In [2]:
import random
random.seed(42)

In [3]:
from string import punctuation


def to_lower(review):
	return str.lower(review)


def no_punct(review):
	return review.translate(str.maketrans('', '', punctuation))


def to_fasttext(review, label):

	label_str = 'positive'
	if label == 0:
		label_str = 'negative'
	return '__label__' + label_str + " " + review


def pretreatment(dataset):
	labels = dataset['label']
	reviews = dataset['text']
	corrected_review_list = []
	for i in range(len(reviews)):
		lower = to_lower(reviews[i])
		no_punc = no_punct(lower)
		corrected_review = to_fasttext(no_punc, labels[i])
		corrected_review_list.append(corrected_review)
	return corrected_review_list


X_treated = pretreatment(dataset_train)
with open("X_train.txt", mode="w") as file:
	file.write("\n".join(X_treated))


In [4]:
model = fasttext.train_supervised(input="X_train.txt")

Read 5M words
Number of words:  121891
Number of labels: 2
Progress: 100.0% words/sec/thread: 2204108 lr:  0.000000 avg.loss:  0.380869 ETA:   0h 0m 0s


In [9]:
dataset_test = load_dataset("imdb", split="test")
X_test_treated = pretreatment(dataset_test)
X_test_treated[:5]

Reusing dataset imdb (/home/victor/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


['__label__negative i love scifi and am willing to put up with a lot scifi moviestv are usually underfunded underappreciated and misunderstood i tried to like this i really did but it is to good tv scifi as babylon 5 is to star trek the original silly prosthetics cheap cardboard sets stilted dialogues cg that doesnt match the background and painfully onedimensional characters cannot be overcome with a scifi setting im sure there are those of you out there who think babylon 5 is good scifi tv its not its clichéd and uninspiring while us viewers might like emotion and character development scifi is a genre that does not take itself seriously cf star trek it may treat important issues yet not as a serious philosophy its really difficult to care about the characters here as they are not simply foolish just missing a spark of life their actions and reactions are wooden and predictable often painful to watch the makers of earth know its rubbish as they have to always say gene roddenberrys ea

In [6]:
with open("X_test.txt", mode="w") as file:
	file.write("\n".join(X_test_treated))
model.test("X_test.txt")


(25000, 0.8678, 0.8678)

In [7]:
model.test("X_train.txt")

(25000, 0.89524, 0.89524)

In [8]:
fasttext.version

AttributeError: module 'fasttext' has no attribute '__version__'

In [ ]:
!pip ver